In [ ]:
from __future__ import absolute_import, division, print_function
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import pickle
import csv
import random
import tensorflow as tf
from tensorflow.keras import Sequential
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, LSTM, Embedding, Reshape
print("Version: ", tf.__version__)

In [ ]:
url="https://raw.githubusercontent.com/tlkh/text-emotion-classification/master/data.csv"
import urllib.request
urllib.request.urlretrieve(url, "data.csv")
sentence=[]
sentiment=[]
texts=[]
with open("data.csv") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if row not in texts:
            texts.append(row)
random.shuffle(texts)
print(texts[12])
for row in texts:
    sentence.append(row[0])
    sentiment.append(int(row[1]))
# neutral, happy, sad, anger, hate.
print(len(sentence))

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(sentence)
from tensorflow.keras.preprocessing.sequence import pad_sequences
sentence_token = tokenizer.texts_to_sequences(sentence)
train_x = pad_sequences(sentence_token[:40000], maxlen=20)
train_y= np.array(sentiment[:40000])
test_x = pad_sequences(sentence_token[40000:], maxlen=20)
test_y=np.array(sentiment[40000:])
print(train_x.shape)

In [ ]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D
tf.keras.backend.clear_session()
model = Sequential()
model.add(Embedding(20000, 128))
# model.add(tf.keras.layers.Conv1D(
#                  100,
#                  5,
#                  padding='valid',
#                  activation='relu',
#                  strides=1))
model.add(LSTM(50))
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='softmax'))
optim=tf.keras.optimizers.Adam(
    learning_rate=0.0001, beta_1=0.9, beta_2=0.9999)
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=optim,
              metrics=['accuracy'])
print(model.summary())
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(train_x, train_y , batch_size=32, epochs=4, validation_data=(test_x, test_y), callbacks=[callback])

In [ ]:
output = ["neutral", "happy", "sad", "anger", "hate"]
text="how to sleep"
text_p = tokenizer.texts_to_sequences([text])
test = pad_sequences(text_p, maxlen=20)
print(text)
print(text,output[np.argmax(model.predict(pad_sequences(test)))])

In [ ]:
model.save('Model_5.h5', save_format='tf')